5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

(a) Fit a logistic regression model that uses income and balance to predict default.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# 데이터셋 로드 및 Unnamed 열 제거
data = pd.read_csv("Default.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# 독립 변수(X)와 종속 변수(y) 설정
X = data[['income', 'balance']]
y = data['default'].apply(lambda x: 1 if x == 'Yes' else 0)  # Yes를 1, No를 0으로 변환

# 데이터 분할: 70% 훈련 세트, 30% 테스트 세트
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 로지스틱 회귀 모델 초기화 및 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 테스트 세트로 예측
y_pred = model.predict(X_test)

# 모델 평가
print("모델 정확도:", accuracy_score(y_test, y_pred))
print("\n분류 리포트:\n", classification_report(y_test, y_pred))


모델 정확도: 0.9733333333333334

분류 리포트:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      2906
           1       0.71      0.26      0.38        94

    accuracy                           0.97      3000
   macro avg       0.84      0.63      0.68      3000
weighted avg       0.97      0.97      0.97      3000



(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터셋 로드 및 Unnamed 열 제거
data = pd.read_csv("Default.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# 독립 변수(X)와 종속 변수(y) 설정
X = data[['income', 'balance']]
y = data['default'].apply(lambda x: 1 if x == 'Yes' else 0)  # Yes를 1, No를 0으로 변환

# 데이터를 훈련 세트와 검증 세트로 분할 (70% 훈련, 30% 검증)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# 분할된 데이터 크기 확인
print("훈련 세트 크기:", X_train.shape[0])
print("검증 세트 크기:", X_val.shape[0])


훈련 세트 크기: 7000
검증 세트 크기: 3000


ii. Fit a multiple logistic regression model using only the training observations.

In [4]:
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 초기화 및 학습 (훈련 세트만 사용)
model = LogisticRegression()
model.fit(X_train, y_train)

# 학습이 완료된 모델의 계수 확인
print("모델 회귀 계수:", model.coef_)
print("모델 절편:", model.intercept_)

모델 회귀 계수: [[1.87209322e-05 5.72532879e-03]]
모델 절편: [-11.60187871]


iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

In [5]:
# 검증 세트에서 default 상태의 posterior probability 계산
y_prob = model.predict_proba(X_val)[:, 1]  # default일 확률 추출

# posterior probability가 0.5보다 크면 1, 아니면 0으로 변환하여 예측 결과 생성
y_pred = (y_prob > 0.5).astype(int)

# 예측된 값 확인
print("검증 세트 예측 결과 (일부):", y_pred[:10])  # 일부 예측값 출력
print("검증 세트의 posterior probability (일부):", y_prob[:10])  # 일부 확률 출력

검증 세트 예측 결과 (일부): [0 0 0 0 0 0 0 0 0 0]
검증 세트의 posterior probability (일부): [5.77365597e-02 1.66835359e-03 1.37619425e-05 3.23932676e-05
 5.25563312e-02 2.81716941e-02 3.45383390e-01 2.00226893e-03
 4.25446212e-02 8.26091888e-03]


iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [6]:
from sklearn.metrics import accuracy_score

# 검증 세트 오류율 계산
validation_error = 1 - accuracy_score(y_val, y_pred)

# 오류율 출력
print("Validation Set Error:", validation_error)

Validation Set Error: 0.026666666666666616


(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# 데이터셋 로드 및 Unnamed 열 제거
data = pd.read_csv("Default.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# 독립 변수(X)와 종속 변수(y) 설정
X = data[['income', 'balance']]
y = data['default'].apply(lambda x: 1 if x == 'Yes' else 0)  # Yes를 1, No를 0으로 변환

# 세 번의 분할을 위한 결과 저장
validation_errors = []

# 세 번의 반복
for i in range(3):
    print(f"\n반복 {i+1}:")
    
    # Step i: 데이터를 훈련 세트와 검증 세트로 분할 (매 반복마다 새로운 랜덤 분할)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=np.random.randint(10000))
    print("훈련 세트 크기:", X_train.shape[0])
    print("검증 세트 크기:", X_val.shape[0])

    # Step ii: 로지스틱 회귀 모델 학습 (훈련 세트만 사용)
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # 모델 회귀 계수와 절편 확인 (각 반복마다 다를 수 있음)
    print("모델 회귀 계수:", model.coef_)
    print("모델 절편:", model.intercept_)

    # Step iii: 검증 세트에 대해 default 상태 예측
    y_prob = model.predict_proba(X_val)[:, 1]
    y_pred = (y_prob > 0.5).astype(int)

    # Step iv: 검증 세트 오류율 계산 및 출력
    validation_error = 1 - accuracy_score(y_val, y_pred)
    validation_errors.append(validation_error)
    print("Validation Set Error:", validation_error)

# 전체 결과 요약
print("\n세 번의 반복에서의 평균 검증 오류율:", np.mean(validation_errors))
print("세 번의 반복에서의 검증 오류율:", validation_errors)



반복 1:
훈련 세트 크기: 7000
검증 세트 크기: 3000
모델 회귀 계수: [[2.03106318e-05 5.61286183e-03]]
모델 절편: [-11.51826221]
Validation Set Error: 0.027000000000000024

반복 2:
훈련 세트 크기: 7000
검증 세트 크기: 3000
모델 회귀 계수: [[2.68603723e-05 5.49184192e-03]]
모델 절편: [-11.54774659]
Validation Set Error: 0.02733333333333332

반복 3:
훈련 세트 크기: 7000
검증 세트 크기: 3000
모델 회귀 계수: [[1.80522967e-05 5.37432181e-03]]
모델 절편: [-10.96919818]
Validation Set Error: 0.021666666666666612

세 번의 반복에서의 평균 검증 오류율: 0.02533333333333332
세 번의 반복에서의 검증 오류율: [0.027000000000000024, 0.02733333333333332, 0.021666666666666612]


(d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 데이터셋 로드 및 Unnamed 열 제거
data = pd.read_csv("Default.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# student 변수를 더미 변수로 변환 (Yes -> 1, No -> 0)
data['student'] = data['student'].apply(lambda x: 1 if x == 'Yes' else 0)

# 독립 변수(X)와 종속 변수(y) 설정
X = data[['income', 'balance', 'student']]
y = data['default'].apply(lambda x: 1 if x == 'Yes' else 0)  # Yes를 1, No를 0으로 변환

# 데이터를 훈련 세트와 검증 세트로 분할 (70% 훈련, 30% 검증)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# 로지스틱 회귀 모델 학습 (훈련 세트만 사용)
model = LogisticRegression()
model.fit(X_train, y_train)

# 검증 세트에 대한 예측 및 오류율 계산
y_pred = model.predict(X_val)
validation_error = 1 - accuracy_score(y_val, y_pred)

# 결과 출력
print("모델 회귀 계수:", model.coef_)
print("모델 절편:", model.intercept_)
print("Validation Set Error:", validation_error)


모델 회귀 계수: [[ 5.82361683e-06  5.78857147e-03 -4.67458505e-01]]
모델 절편: [-11.10816439]
Validation Set Error: 0.026666666666666616
